# TOP 1000 filmes e séries de TV - IMDb

## Projeto Final - Banco de Dados I

- Escolha um dataset de sua preferência (Kaggle, UCI,...);
  - https://www.kaggle.com/datasets;
  - https://archive.ics.uci.edu/datasets;
  - https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html;
  - https://dados.gov.br/;
- Coloque-o no postgres; 
- Usando o Jupyter notebook, faça uma análise descritiva;
    - Qual as tipagens das colunas?;
    - Quantos nulos temos em cada coluna?;
    - Quantos valores únicos temos nas colunas categoricas?;
    - ...;
- Ainda no Jupyter, faça 10 perguntas ao seu dataset e responda-as com querys;
    - Qual o top 10 vendas?;
    - Qual o top 5 clientes?;
    - Qual medida é minoria?;
    - ...;

### Integrantes

- Guilherme Araújo Mendes de Souza
- Laura Busin Campos

### Dataset utilizado
- **IMDB Movies Dataset:** https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

### Importação e verificação da base

#### Bibliotecas

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text

#### Verificação do estado da base

In [115]:
imdb = pd.read_csv('../Data/imdb_top_1000.csv')

In [116]:
imdb.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [117]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Poster_Link    1000 non-null   object 
 1   Series_Title   1000 non-null   object 
 2   Released_Year  1000 non-null   object 
 3   Certificate    899 non-null    object 
 4   Runtime        1000 non-null   object 
 5   Genre          1000 non-null   object 
 6   IMDB_Rating    1000 non-null   float64
 7   Overview       1000 non-null   object 
 8   Meta_score     843 non-null    float64
 9   Director       1000 non-null   object 
 10  Star1          1000 non-null   object 
 11  Star2          1000 non-null   object 
 12  Star3          1000 non-null   object 
 13  Star4          1000 non-null   object 
 14  No_of_Votes    1000 non-null   int64  
 15  Gross          831 non-null    object 
dtypes: float64(2), int64(1), object(13)
memory usage: 125.1+ KB


#### Tratamento do tipo de algumas colunas

In [118]:
imdb['Gross'] = imdb['Gross'].str.replace(',', '').replace('', '0').astype(float)

In [119]:
imdb['Runtime'] = imdb['Runtime'].str.replace(' min', '').replace('', '0').astype(int)

#### Separação da coluna de Gênero em outras colunas

In [120]:
imdb["Genre"] = imdb["Genre"].str.split(', ')
imdb = imdb.join(imdb['Genre'].apply(lambda x: pd.Series(x)).add_prefix('Genre_'))

#### Dropando colunas indesejadas

In [121]:
imdb.drop(columns = ["Genre", "Poster_Link"], inplace = True)

#### Verificando o estado da base depois do tratamento

In [122]:
imdb.head()

,Series_Title,Released_Year,Certificate,Runtime,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Genre_0,Genre_1,Genre_2
0,The Shawshank Redemption,1994,A,142,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0,Drama,NaN,NaN
1,The Godfather,1972,A,175,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0,Crime,Drama,NaN
2,The Dark Knight,2008,UA,152,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0,Action,Crime,Drama
3,The Godfather: Part II,1974,A,202,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0,Crime,Drama,NaN
4,12 Angry Men,1957,U,96,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0,Crime,Drama,NaN


In [123]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Certificate    899 non-null    object 
 3   Runtime        1000 non-null   int32  
 4   IMDB_Rating    1000 non-null   float64
 5   Overview       1000 non-null   object 
 6   Meta_score     843 non-null    float64
 7   Director       1000 non-null   object 
 8   Star1          1000 non-null   object 
 9   Star2          1000 non-null   object 
 10  Star3          1000 non-null   object 
 11  Star4          1000 non-null   object 
 12  No_of_Votes    1000 non-null   int64  
 13  Gross          831 non-null    float64
 14  Genre_0        1000 non-null   object 
 15  Genre_1        895 non-null    object 
 16  Genre_2        646 non-null    object 
dtypes: float64(3), int32(1), int64(1), object(12)
memory 

#### Salvando o Data Frame em um csv

In [124]:
imdb.to_csv('../Data/imdb_top_1000_tratado.csv', index=False)

### Criação da tabela no PostgreSQL para fazer a importação do csv criado

CREATE TABLE ada.imdb(
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;id&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;SERIAL
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,nm_filme&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(100)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,ano&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(4)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,certificado&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,duracao&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;INTEGER
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,classificacao_imdb&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FLOAT
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,resumo&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(1000)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,pontuacao&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FLOAT
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,diretor&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,estrela4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(50)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,votos&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;INTEGER
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,bilheteria&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;FLOAT
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,genero1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,genero2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,genero3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;VARCHAR(15)
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,PRIMARY KEY (id)
<br>);

#### Configurando a conexão com o banco de dados

Para fazer a conexão com o seu banco de dados, salve um arquivo com o nome configSGBD.txt fora da pasta do projeto com as seguintes inforamções e fotrmatação:
- {"SGBD": "xxxxx", "USER": "xxxxx", "SENHA": "xxxxx", "HOST": "xxxxx", "DATABASE": "xxxxx"}

In [127]:
# Puxando a informação do arquivo de configuração do SGBD

config = ''
with open('..\..\configSGBD.txt', 'r') as file:
    for line in file:
        config += line.strip('')

In [128]:
# Transformando em um dicionário

dict_config = eval(config)

In [138]:
# Fazendo a conexão com o SGBD

engine = create_engine(f"{dict_config['SGBD']}://{dict_config['USER']}:{dict_config['SENHA']}@{dict_config['HOST']}:5432/{dict_config['DATABASE']}")
conn = engine.connect()

In [12]:
# Verificando se a conexão deu certo

pd.read_sql_query(text("SELECT * FROM ada.imdb LIMIT 5"), con = conn)

,id,nm_filme,ano,certificado,duracao,classificacao_imdb,resumo,pontuacao,diretor,estrela1,estrela2,estrela3,estrela4,votos,bilheteria,genero1,genero2,genero3
0,1,The Shawshank Redemption,1994,A,142,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0,Drama,None,None
1,2,The Godfather,1972,A,175,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0,Crime,Drama,None
2,3,The Dark Knight,2008,UA,152,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0,Action,Crime,Drama
3,4,The Godfather: Part II,1974,A,202,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0,Crime,Drama,None
4,5,12 Angry Men,1957,U,96,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0,Crime,Drama,None


### Análise Descritiva

#### Tamanho da amostra

In [130]:
query = """
SELECT 
    COUNT(*) AS "Tamanho da amostra"
FROM ada.imdb;
"""

pd.read_sql_query(text(query), con = conn)

,Tamanho da amostra
0,1000


#### Quais são os tipos de dados

In [131]:
query = """
SELECT
    column_name, 
    DATA_TYPE
FROM
    information_schema.columns
WHERE
    table_name = 'imdb' AND table_schema = 'ada'
"""

pd.read_sql_query(text(query), con = conn)

,column_name,data_type
0,id,integer
1,nm_filme,character varying
2,ano,character varying
3,certificado,character varying
4,duracao,integer
5,classificacao_imdb,double precision
6,resumo,character varying
7,pontuacao,double precision
8,diretor,character varying
9,estrela1,character varying


#### Quantos termos nulos em cada coluna

In [132]:
query = """
SELECT 
    SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END)                     AS "ID",
    SUM(CASE WHEN nm_filme IS NULL THEN 1 ELSE 0 END)               AS "Nome do filme",
    SUM(CASE WHEN ano IS NULL THEN 1 ELSE 0 END)                    AS "Ano",
    SUM(CASE WHEN certificado IS NULL THEN 1 ELSE 0 END)            AS "Certificado",
    SUM(CASE WHEN duracao IS NULL THEN 1 ELSE 0 END)                AS "Duracao",
    SUM(CASE WHEN classificacao_imdb IS NULL THEN 1 ELSE 0 END)     AS "Nota IMDB",
    SUM(CASE WHEN resumo IS NULL THEN 1 ELSE 0 END)                 AS "Resumo",
    SUM(CASE WHEN pontuacao IS NULL THEN 1 ELSE 0 END)              AS "Pontuacao",
    SUM(CASE WHEN diretor IS NULL THEN 1 ELSE 0 END)                AS "Diretor",
    SUM(CASE WHEN estrela1 IS NULL THEN 1 ELSE 0 END)               AS "Ator ou Atriz",
    SUM(CASE WHEN estrela2 IS NULL THEN 1 ELSE 0 END)               AS "Ator2 ou Atriz2",
    SUM(CASE WHEN estrela3 IS NULL THEN 1 ELSE 0 END)               AS "Ator3 ou Atriz3",
    SUM(CASE WHEN estrela4 IS NULL THEN 1 ELSE 0 END)               AS "Ator4 ou Atriz4",
    SUM(CASE WHEN votos IS NULL THEN 1 ELSE 0 END)                  AS "Votos",
    SUM(CASE WHEN bilheteria IS NULL THEN 1 ELSE 0 END)             AS "Bilheteria",
    SUM(CASE WHEN genero1 IS NULL THEN 1 ELSE 0 END)                AS "Gênero1",
    SUM(CASE WHEN genero2 IS NULL THEN 1 ELSE 0 END)                AS "Gênero2",
    SUM(CASE WHEN genero3 IS NULL THEN 1 ELSE 0 END)                AS "Gênero3"
FROM ada.imdb;
"""

pd.read_sql_query(text(query), con = conn)

,ID,Nome do filme,Ano,Certificado,Duracao,Nota IMDB,Resumo,Pontuacao,Diretor,Ator ou Atriz,Ator2 ou Atriz2,Ator3 ou Atriz3,Ator4 ou Atriz4,Votos,Bilheteria,Gênero1,Gênero2,Gênero3
0,0,0,0,101,0,0,0,157,0,0,0,0,0,0,169,0,105,354


#### Média das colunas numéricas

In [145]:
# Fazendo a conexão com o SGBD

engine = create_engine(f"{dict_config['SGBD']}://{dict_config['USER']}:{dict_config['SENHA']}@{dict_config['HOST']}:5432/{dict_config['DATABASE']}")
conn = engine.connect()

In [22]:
#Média
query = """
SELECT
    AVG(duracao)            AS "Duração",
    AVG(votos)              AS "Votos",
    AVG(classificacao_imdb) AS "Notas IMDB",
    AVG(bilheteria)         AS "Bilheteria",
    AVG(pontuacao)          AS "pontuacao"
FROM ada.imdb   
"""

pd.read_sql_query(text(query), con = conn)

,Duração,Votos,Notas IMDB,Bilheteria,pontuacao
0,122.891,273692.911,7.9493,5.653688e+07,65.73


#### Quantos valores únicos em cada coluna

In [25]:
query = """
SELECT 
    COUNT(DISTINCT id)                  AS "ID",
    COUNT(DISTINCT nm_filme)            AS "Nome do filme",
    COUNT(DISTINCT ano)                 AS "Ano",
    COUNT(DISTINCT certificado)         AS "Certificado",
    COUNT(DISTINCT duracao)             AS "Duracao",
    COUNT(DISTINCT classificacao_imdb)  AS "Nota IMDB",
    COUNT(DISTINCT resumo)              AS "Resumo",
    COUNT(DISTINCT pontuacao)           AS "Pontuacao",
    COUNT(DISTINCT diretor)             AS "Diretor",
    COUNT(DISTINCT estrela1)            AS "Ator ou Atriz",
    COUNT(DISTINCT estrela2)            AS "Ator2 ou Atriz2",
    COUNT(DISTINCT estrela3)            AS "Ator3 ou Atriz3",
    COUNT(DISTINCT estrela4)            AS "Ator4 ou Atriz4",
    COUNT(DISTINCT votos)               AS "Votos",
    COUNT(DISTINCT bilheteria)          AS "Bilheteria",
    COUNT(DISTINCT genero1)             AS "Gênero1",
    COUNT(DISTINCT genero2)             AS "Gênero2",
    COUNT(DISTINCT genero3)             AS "Gênero3"
FROM ada.imdb;
"""

pd.read_sql_query(text(query), con = conn)

,ID,Nome do filme,Ano,Certificado,Duracao,Nota IMDB,Resumo,Pontuacao,Diretor,Ator ou Atriz,Ator2 ou Atriz2,Ator3 ou Atriz3,Ator4 ou Atriz4,Votos,Bilheteria,Gênero1,Gênero2,Gênero3
0,1000,999,100,16,140,17,1000,64,548,660,841,891,939,999,824,14,20,19


### 10 perguntas sobre o dataset

#### **1 -** Qual é o Top 10 filmes com maior bilheteria?

In [46]:
query = """
SELECT
    nm_filme                    AS "Nome do filme",
    ano                         AS "Ano",
    classificacao_imdb          AS "Nota IMBD",
    bilheteria/1000000000       AS "Bilheteria"
FROM ada.imdb 
ORDER BY bilheteria DESC
LIMIT 10
"""

pd.read_sql_query(text(query), con = conn)

,Nome do filme,Ano,Nota IMBD,Bilheteria
0,Star Wars: Episode VII - The Force Awakens,2015,7.9,936662225.0
1,Avengers: Endgame,2019,8.4,858373000.0
2,Avatar,2009,7.8,760507625.0
3,Avengers: Infinity War,2018,8.4,678815482.0
4,Titanic,1997,7.8,659325379.0
5,The Avengers,2012,8.0,623279547.0
6,Incredibles 2,2018,7.6,608581744.0
7,The Dark Knight,2008,9.0,534858444.0
8,Rogue One,2016,7.8,532177324.0
9,The Dark Knight Rises,2012,8.4,448139099.0


#### **2 -** Qual é o Top 10 filmes com maior classificação IMDb?

In [55]:
query = """
SELECT
    nm_filme                       AS "Nome do filme",
    ano                            AS "Ano",
    classificacao_imdb             AS "Nota IMBD",
    bilheteria/10000000000         AS "Bilheteria (Bi)"
FROM ada.imdb 
ORDER BY classificacao_imdb DESC
LIMIT 10
"""

pd.read_sql_query(text(query), con = conn)

,Nome do filme,Ano,Nota IMBD,Bilheteria (Bi)
0,The Shawshank Redemption,1994,9.3,0.002834
1,The Godfather,1972,9.2,0.013497
2,The Dark Knight,2008,9.0,0.053486
3,The Godfather: Part II,1974,9.0,0.005730
4,12 Angry Men,1957,9.0,0.000436
5,Pulp Fiction,1994,8.9,0.010793
6,The Lord of the Rings: The Return of the King,2003,8.9,0.037785
7,Schindler's List,1993,8.9,0.009690
8,Fight Club,1999,8.8,0.003703
9,Inception,2010,8.8,0.029258


#### **3 -** Qual é o Top 5 gêneros com maior bilheteria?

In [150]:
query = """
SELECT
    genero1                         AS "Gênero",
    SUM(bilheteria)/10000000000     AS "Bilheteria total (Bi)"
FROM ada.imdb 
GROUP BY genero1
ORDER BY SUM(bilheteria) DESC
LIMIT 5
"""

pd.read_sql_query(text(query), con = conn)

,Gênero,Bilheteria total (Bi)
0,Fantasy,NaN
1,Action,2.001680
2,Drama,0.905048
3,Animation,0.857382
4,Adventure,0.527375


#### **4 -** Qual é o Top 10 diretores com maior bilheteria?

In [51]:
query = """
SELECT
    diretor                         AS "Diretor",
    SUM(bilheteria)/1000000000      AS "Bilheteria (Bi)"
FROM ada.imdb 
GROUP BY diretor
ORDER BY SUM(bilheteria) DESC
LIMIT 10
"""
pd.read_sql_query(text(query), con = conn)

,Diretor,Bilheteria (Bi)
0,Steven Spielberg,2.478133
1,Anthony Russo,2.205039
2,Christopher Nolan,1.937454
3,James Cameron,1.748237
4,Peter Jackson,1.597312
5,J.J. Abrams,1.423171
6,Brad Bird,1.099628
7,Robert Zemeckis,1.049446
8,David Yates,0.978954
9,Pete Docter,0.939382


#### **5 -** Qual o Top 3 gêneros com mais filmes?

In [56]:
query = """
SELECT
    genero1             AS "Gênero",
    COUNT(nm_filme)     AS "Quantidade de filmes"
FROM ada.imdb 
GROUP BY genero1
ORDER BY COUNT(*) DESC
LIMIT 3
"""

pd.read_sql_query(text(query), con = conn)

,Gênero,Quantidade de filmes
0,Drama,289
1,Action,172
2,Comedy,155


#### **6 -** Qual é o Top 3 filmes (avaliação/bilheteria) do gênero com mais filmes?

In [66]:
query = """
SELECT
    nm_filme                AS "Nome do Filme",
    bilheteria/1000000000   AS "Bilheteria (Bi)",
    genero1                 AS "Gênero"
FROM ada.imdb
WHERE genero1 IN (
    SELECT
        genero1
    FROM ada.imdb 
    GROUP BY genero1
    ORDER BY COUNT(*) DESC
    LIMIT 1
) 
GROUP BY genero1, nm_filme, bilheteria
ORDER BY bilheteria DESC
LIMIT 3;
"""

pd.read_sql_query(text(query), con = conn)

,Nome do Filme,Bilheteria (Bi),Gênero
0,Titanic,0.659325,Drama
1,Forrest Gump,0.330252,Drama
2,The Sixth Sense,0.293506,Drama


In [64]:
query = """
SELECT
    nm_filme            AS "Nome do Filme",
    classificacao_imdb  AS "Nota IMBD",
    genero1             AS "Gênero"
FROM ada.imdb
WHERE genero1 IN (
    SELECT
        genero1
    FROM ada.imdb 
    GROUP BY genero1
    ORDER BY COUNT(*) DESC
    LIMIT 1
) 
GROUP BY genero1, nm_filme, classificacao_imdb
ORDER BY classificacao_imdb DESC
LIMIT 3;
"""

pd.read_sql_query(text(query), con = conn)

,Nome do Filme,Nota IMBD,Gênero
0,The Shawshank Redemption,9.3,Drama
1,Fight Club,8.8,Drama
2,Forrest Gump,8.8,Drama


#### **7 -** Qual é Top 5 diretores (avaliação/bilheteria)?

In [69]:
query = """
SELECT
    diretor                        AS "Diretor",
    SUM(bilheteria)/1000000000     AS "Bilheteria (Bi)"
FROM ada.imdb 
GROUP BY diretor
ORDER BY SUM(bilheteria) DESC
LIMIT 5
"""
pd.read_sql_query(text(query), con = conn)

,Diretor,Bilheteria (Bi)
0,Steven Spielberg,2.478133
1,Anthony Russo,2.205039
2,Christopher Nolan,1.937454
3,James Cameron,1.748237
4,Peter Jackson,1.597312


In [70]:
query = """
SELECT
    diretor                     AS "Diretor",
    AVG(classificacao_imdb)     AS "Nota IMBD"
FROM ada.imdb 
GROUP BY diretor
ORDER BY AVG(classificacao_imdb) DESC
LIMIT 5
"""
pd.read_sql_query(text(query), con = conn)

,Diretor,Nota IMBD
0,Frank Darabont,8.95
1,Irvin Kershner,8.70
2,Lana Wachowski,8.70
3,Roberto Benigni,8.60
4,Sudha Kongara,8.60


#### **8 -** Qual é o Top 10 atores com melhor média (avaliação/bilheteria)?

In [71]:
query = """
SELECT
    estrela1                    AS "Ator(a)",
    AVG(classificacao_imdb)     AS "Nota IMBD"
FROM ada.imdb 
GROUP BY estrela1
ORDER BY AVG(classificacao_imdb) DESC
LIMIT 10
"""
pd.read_sql_query(text(query), con = conn)

,Ator(a),Nota IMBD
0,Tim Robbins,9.30
1,John Travolta,8.90
2,Elijah Wood,8.80
3,Lilly Wachowski,8.70
4,Marlon Brando,8.65
5,Suriya,8.60
6,Roberto Benigni,8.60
7,Jodie Foster,8.60
8,Kátia Lund,8.60
9,Lin-Manuel Miranda,8.60


In [73]:
query = """
SELECT
    estrela1                        AS "Ator(a)",
    AVG(bilheteria)/1000000000      AS "Bilheteria (Bi)"
FROM ada.imdb 
GROUP BY estrela1
ORDER BY AVG(bilheteria) DESC
LIMIT 10
"""

pd.read_sql_query(text(query), con = conn)

,Ator(a),Bilheteria (Bi)
0,Daisy Ridley,0.936662
1,Sam Worthington,0.760508
2,Joe Russo,0.551260
3,Felicity Jones,0.532177
4,Henry Thomas,0.435111
5,Craig T. Nelson,0.435011
6,Rob Minkoff,0.422784
7,Robert Downey Jr.,0.383573
8,Lee Unkrich,0.380843
9,Chris Pratt,0.361495


#### **9 -** Qual é a relação votos e atores/diretores

In [25]:
query = """
SELECT
    diretor         AS "Diretor(a)",
    AVG(votos)      AS "Votos"
FROM ada.imdb 
GROUP BY diretor
ORDER BY AVG(votos) DESC
LIMIT 5
"""

pd.read_sql_query(text(query), con = conn)

,Diretor(a),Votos
0,Frank Darabont,1745452.000
1,Lana Wachowski,1676426.000
2,Christopher Nolan,1447293.125
3,George Lucas,1231473.000
4,Peter Jackson,1229715.800


In [135]:
query = """
SELECT
    estrela1        AS "Ator(a)",
    AVG(votos)      AS "Votos"
FROM ada.imdb 
GROUP BY estrela1
ORDER BY AVG(votos) DESC
LIMIT 5
"""

pd.read_sql_query(text(query), con = conn)

,Ator(a),Votos
0,Tim Robbins,2343110.0
1,John Travolta,1826188.0
2,Lilly Wachowski,1676426.0
3,Elijah Wood,1596598.0
4,Jodie Foster,1270197.0


#### **10 -** Qual é o Top 3 generos com as melhores médias de avaliação/bilheteria?

In [74]:
query = """
SELECT
    genero1                     AS "Gênero",
    AVG(classificacao_imdb)     AS "Nota IMBD"
FROM ada.imdb 
GROUP BY genero1
ORDER BY AVG(classificacao_imdb) DESC
LIMIT 3
"""

pd.read_sql_query(text(query), con = conn)

,Gênero,Nota IMBD
0,Western,8.350000
1,Crime,8.016822
2,Fantasy,8.000000


In [75]:
query = """
SELECT
    genero1                        AS "Gênero",
    AVG(bilheteria) /1000000000    AS "Bilheteria (Bi)"
FROM ada.imdb 
GROUP BY genero1
ORDER BY AVG(bilheteria) DESC
LIMIT 3
"""

pd.read_sql_query(text(query), con = conn)

,Gênero,Bilheteria (Bi)
0,Family,0.219555
1,Action,0.116377
2,Animation,0.104559


### Perguntas extras

#### Qual o filme mais assistido em 2001?

In [76]:
query = """
SELECT
    nm_filme        AS "Nome do Filme",
    bilheteria      AS "Bilheteria",
    ano             AS "Ano"
FROM ada.imdb
WHERE ano = '2001'
ORDER BY bilheteria DESC
LIMIT 1
"""

pd.read_sql_query(text(query), con = conn)

,Nome do Filme,Bilheteria,Ano
0,Harry Potter and the Sorcerer's Stone,317575550.0,2001


#### Quantos filmes foram lançados em cada ano (de 1990 a 2000)?

In [87]:
query = """
SELECT
    ano                 AS "Ano"
    ,COUNT(nm_filme)    AS "Qtd de filmes"
FROM ada.imdb
WHERE ano BETWEEN '1990' AND '2000'
GROUP BY ano
ORDER BY ano DESC
"""

pd.read_sql_query(text(query), con = conn)

,Ano,Qtd de filmes
0,2000,19
1,1999,17
2,1998,17
3,1997,19
4,1996,10
5,1995,19
6,1994,13
7,1993,23
8,1992,12
9,1991,12


#### Quantos filmes possuem um, dois ou três gêneros entre os anos 2000 e 2010?

In [111]:
query = """
SELECT
    ano                                                                             AS "Ano"
    ,SUM(CASE WHEN (genero2 IS NULL) AND( genero3 IS NULL) THEN 1 ELSE 0 END)       AS "Filmes com 1 genero"
    ,SUM(CASE WHEN (genero2 IS NOT NULL) AND (genero3 IS NULL) THEN 1 ELSE 0 END)   AS "Filmes com 2 generos"
    ,SUM(CASE WHEN (genero3 IS NOT NULL) THEN 1 ELSE 0 END)                         AS "Filmes com 3 generos"
FROM ada.imdb
WHERE ano BETWEEN '2000' AND '2010'
GROUP BY ano
ORDER BY ano
"""

pd.read_sql_query(text(query), con = conn)

,Ano,Filmes com 1 genero,Filmes com 2 generos,Filmes com 3 generos
0,2000,1,5,13
1,2001,2,6,19
2,2002,3,3,13
3,2003,2,5,15
4,2004,2,7,22
5,2005,1,6,10
6,2006,1,4,21
7,2007,4,2,20
8,2008,3,5,13
9,2009,1,4,24


#### Qual a média de bilheteria dos filmes lançados nos séculos 19 e 20?

In [148]:
query = """
SELECT
    LEFT(ano, 2)                                            AS "Século"
    ,ROUND(CAST(AVG(classificacao_imdb) AS NUMERIC), 2)     AS "Nota média"
FROM ada.imdb
WHERE ano <> 'PG'
GROUP BY LEFT(ano, 2)
ORDER BY LEFT(ano, 2)
"""

pd.read_sql_query(text(query), con = conn)

,Século,Nota média
0,19,7.98
1,20,7.91
